In [100]:
import requests
from bs4 import BeautifulSoup, SoupStrainer
import time
import pandas as pd
import numpy as np
import glob

In [101]:
# Pull in results from previous webscraping to get drugs
medsDF = pd.read_csv('Medications_SideEffects.csv', index_col=0)
medsDF = medsDF.dropna(axis=0, how='all', subset=['More common', 'Less common'])
medsDF = medsDF.fillna('')

In [102]:
medsDF

,Condition,Medication,Alternate names,More common,Less common
0,ADHD,Methylphenidate,"Methylin, Ritalin, Concerta",Headache; loss of appetite; nervousness; stoma...,Anger; decreased appetite; dizziness; drowsine...
2,ADHD,Daytrana,,Headache; loss of appetite; nervousness; stoma...,Anger; decreased appetite; dizziness; drowsine...
3,ADHD,Dexmethylphenidate,"Focalin XR, SR Focalin",Acid or sour stomach; belching; heartburn; ind...,Twitching;
5,ADHD,Amphetamine,"Adderall, Adderall XR",Dry mouth; loss of appetite; sore throat; stom...,Belching; dizziness; heartburn; indigestion; s...
6,ADHD,Lisdexamfetemine,"Vyvanse, Dimesylate",Decreased appetite; headache; nausea; upper ab...,Crying; depersonalization; dry mouth; dysphori...
7,ADHD,Atomoxetine,Strattera,Acid or sour stomach; belching; bleeding betwe...,Abnormal dreams; abnormal orgasm; back pain; b...
8,ADHD,Guanfacine,Intuniv,Constipation; dizziness; dryness of the mouth;...,Belching; decreased appetite; decreased sexual...
9,ADHD,Clonidine Hydrochloride,Kapvay,Constipation;,Darkening of the skin; decreased sexual abilit...
13,ADHD,Bupropion,"Wellbutrin, Wellbutrin XL, Welbutrin SR",Constipation; decrease in appetite; dizziness;...,Blurred vision; change in sense of taste; drow...
14,ADHD,Venlafazine,"Effexor, Effexor XR",Abnormal dreams; chills; constipation; decreas...,Change in taste; muscle tension; yawning;


In [103]:
# Next, we want to pull side effects from drugs.com
def searchWebMD(searchterm, rootUrl='https://www.webmd.com/drugs/2/search?type=drugs&query='):
    if searchterm.find(' ') != -1:
        searchterm.replace(' ', '-')
    response = requests.get(rootUrl+searchterm)
    
    soup = BeautifulSoup(response.content, 'html.parser')
    

    content = soup.find('p', attrs={'class':'no-matches'})
    if content:
        return False, False
    else:
        links = []
        medications = []
        searchLink = False
        for link in soup.find_all('a', href=True):
            if str(link).find('result_') != -1:
                med = str(link)
                med = med[med.find('details">')+9:med.rfind('</a>')]
                medications.append(med)
                
                searchLink = str(link)
                searchLink = searchLink[searchLink.find('href="')+6:searchLink.rfind('"')]
                searchLink = 'https://www.webmd.com'+searchLink
                links.append(searchLink)

        return links, medications


In [104]:
webMDParent = []
webMDExactMeds = []
for ind, medication in zip(medsDF.index, medsDF['Medication']):
    AllLinks = []
    AllExact = []
    searchLinks, exactMeds = searchWebMD(medication)
    if searchLinks:
        print(searchLinks)
        for sl, em in zip(searchLinks, exactMeds):
            AllLinks.append(sl)
            AllExact.append(em)
    if not searchLinks or medsDF.loc[ind]['Alternate names']: 
        if medsDF.loc[ind]['Alternate names']:
            for name in medsDF.loc[ind]['Alternate names'].split(','):
                name = name.replace(' ','')
                searchLinks, exactMeds = searchWebMD(name)
                if searchLinks:
                    print(searchLinks)
                    for sl, em in zip(searchLinks, exactMeds):
                        AllLinks.append(sl)
                        AllExact.append(em)
                time.sleep(1)
            if not searchLinks: 
                AllLinks.append(np.NaN)
                AllExact.append(np.NaN)
        else:
            AllLinks.append(np.NaN)
            AllExact.append(np.NaN)
    
    if len(AllLinks) > 1:
        while np.NaN in AllLinks: AllLinks.remove(np.NaN)
        while np.NaN in AllExact: AllExact.remove(np.NaN)
    elif np.NaN in AllLinks:
        AllLinks = np.NaN
        AllExact = np.NaN
        
    webMDParent.append(AllLinks)
    webMDExactMeds.append(AllExact)
    time.sleep(1)
    
medsDF['Parent links'] = webMDParent
medsDF['Exact medications'] = webMDExactMeds

['https://www.webmd.com/drugs/2/drug-19735/methylphenidate-powder/details', 'https://www.webmd.com/drugs/2/drug-12114-94/methylphenidate-hcl/details', 'https://www.webmd.com/drugs/2/drug-12114-4094/methylphenidate-hcl-cd/details', 'https://www.webmd.com/drugs/2/drug-12114-439/methylphenidate-hcl-solution/details', 'https://www.webmd.com/drugs/2/drug-12114-438/methylphenidate-hcl-tablet-chewable/details', 'https://www.webmd.com/drugs/2/drug-12114-8094/methylphenidate-er-tablet/details', 'https://www.webmd.com/drugs/2/drug-144064-641/methylphenidate-patch-24-hours/details', 'https://www.webmd.com/drugs/2/drug-12114-5094/methylphenidate-er-tablet-24-hr/details', 'https://www.webmd.com/drugs/2/drug-173889-1850/methylphenidate-tablet-disintegrating-er-biphasic-24-hr-tablet-disintegrating-er-hr/details', 'https://www.webmd.com/drugs/2/drug-12114-1516/methylphenidate-suspension-24-hr-reconstituted-suspension-er-reconstituted/details']
['https://www.webmd.com/drugs/2/drug-16869-439/methylin/de

['https://www.webmd.com/drugs/2/drug-91490-404/duloxetine-hcl/details', 'https://www.webmd.com/drugs/2/drug-166394/duloxetine-hcl-bulk-100-powder/details', 'https://www.webmd.com/drugs/2/drug-91490-2114/duloxetine-capsule-delayed-release-sprinkle/details']
['https://www.webmd.com/drugs/2/drug-13507-7155/bupropion-xl/details', 'https://www.webmd.com/drugs/2/drug-13507-155/bupropion-hcl/details', 'https://www.webmd.com/drugs/2/drug-171383-9155/bupropion-hcl-sr/details', 'https://www.webmd.com/drugs/2/drug-89414/bupropion-bulk-100-powder/details', 'https://www.webmd.com/drugs/2/drug-166972-1645/naltrexone-bupropion-tablet-er/details', 'https://www.webmd.com/drugs/2/drug-13507-8155/bupropion-hcl-sr/details', 'https://www.webmd.com/drugs/2/drug-148145/bupropion-dietary-supp-no-15-combination-package/details', 'https://www.webmd.com/drugs/2/drug-148146/bupropion-dietary-supp-no-16-combination-package/details', 'https://www.webmd.com/drugs/2/drug-151959-7155/bupropion-hbr-tablet-er-24-hr/deta

In [ ]:
medsDF = medsDF.dropna(axis=0, how='all', subset=['Parent links'])
medsDF[medsDF['Condition'].eq('Bipolar-Disorder')]

In [106]:
# My data is biased towards chemical name, searching for brand name to get more reviews
def check4brandname(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    content = soup.find('div', attrs={'class':'drug-names'})
    brandname = ''
    if content:
        for child in content.findChildren():
            if str(child)[:3] == '<p>' and str(child).find('COMMON') != -1:
                brandname = str(child)
                brandname = brandname[brandname.find(':')+2:brandname.rfind('</p>')]
                if brandname.find(','):
                    brandname = brandname.replace(' ','')
                    brandname = brandname.split(',')
                break
    return brandname
                
for ind, medication, link in zip(medsDF.index, medsDF['Medication'], medsDF['Parent links']):
    brandname = check4brandname(link[0])
    
    if brandname:
        for name in brandname:
            print(medication, name)
            blank_row = medsDF.loc[ind]
            searchLinks, exactMeds = searchWebMD(medication)
            if not searchLinks: 
                searchLinks = np.NaN
                exactMeds = np.NaN
            row_copy = {'Condition': medsDF.loc[ind]['Condition'],
                        'Medication': name,
                        'Alternate names': medication,
                        'More common': medsDF.loc[ind]['More common'],
                        'Less common': medsDF.loc[ind]['Less common'],
                        'Parent links': searchLinks,
                        'Exact medications': exactMeds}
            medsDF = medsDF.append(row_copy, ignore_index=True)
    
    # Search for brandname links, add to dataframe
    time.sleep(1)

Dexmethylphenidate  Focalin
Amphetamine  Adderall
Guanfacine  Tenex
Clonidine Hydrochloride  Catapres
Bupropion  Aplenzin
Bupropion  WellbutrinXL
Valproic acid  Depakene
carbamazepine Tegretol
Lamotrigine Lamictal
quetiapine Seroquel
fluoxetine Prozac
fluoxetine Sarafem
Fluoxetine Prozac
Fluoxetine Sarafem
Sertraline Zoloft
Paroxetine Paxil
Duloxetine Cymbalta
Bupropion Aplenzin
Bupropion WellbutrinXL
Mirtazapine RemeronSoltab
Aripiprazole  Abilify
Quetiapine Seroquel
Phenelzine Nardil
Selegiline patch  Emsam
Aripiprazole Abilify
Asenapine Saphris
Olanzapine Zyprexa
Paliperidone  Invega
Risperidone  Risperdal
Quetiapine Seroquel
Ziprasidone Geodon


In [109]:
#medsDF[medsDF['Condition'].eq('ADHD')]
medsDF = medsDF.dropna(axis=0, how='all', subset=['Parent links'])

In [110]:
medsDFbackup = medsDF.copy(deep=True)
medsDFbackup2 = medsDF.copy(deep=True)

In [111]:
# Dropping down to combine conditions for some meds
droppedInds = []
for ind in medsDF.index:
    for ind2 in medsDF.index:
        if ind2 not in droppedInds and ind not in droppedInds:
            if medsDF.loc[ind]['Medication'].lower() == medsDF.loc[ind2]['Medication'].lower():
                if medsDF.loc[ind]['Condition'] != medsDF.loc[ind2]['Condition']:
                    cond1 = medsDF.loc[ind]['Condition']            
                    cond2 = medsDF.loc[ind2]['Condition']
                    medsDFbackup.loc[ind]['Condition'] = '{:s}, {:s}'.format(cond1, cond2)
                    medsDFbackup = medsDFbackup.drop(index=ind2)
                    droppedInds.append(ind2)

In [112]:
medsDF = medsDFbackup
medsDF.to_csv('Medications_SideEffects_brandAndlinks.csv')

In [113]:
def pull_review(userPost, revnum=1):
    info = {}
    
    # Pulling basic post info
    info['conditionInfo'] = userPost.find('div', attrs={'class':'conditionInfo'}).text.replace('\r\n\t\t\t\t\t','')
    info['date'] = userPost.find('div', attrs={'class':'date'}).text
    info['reviewer'] = userPost.find('p', attrs={'class':'reviewerInfo'}).text
    
    # Pulling stars info
    content = userPost.find('div', attrs={'id':'ctnStars'})
    catsAndScores = []
    for child in content.findChildren():
        nextisScore=False
        for grandChild in child.findChildren():
            if nextisScore:
                score = grandChild.find('span', attrs={'class':'current-rating'}).text
                score = score[score.rfind(' ')+1:]
                catsAndScores.append((category, score))
                nextisScore=False
            if str(grandChild).find('category') != -1:
                category = str(grandChild)
                category = category[category.find('gory">')+6:category.rfind('</p>')]
                nextisScore=True
    catsAndScores = dict(catsAndScores)
    info['Effectiveness'] = catsAndScores['Effectiveness']
    info['Satisfaction'] = catsAndScores['Satisfaction']
    info['Ease of Use'] = catsAndScores['Ease of Use']
    
    # Pulling comment and cleaning it up
    text = userPost.find('p', attrs={'style':'display:none',
                                     'id':'comFull{:g}'.format(revnum)}).text
    text = text.replace('<strong>Comment:</strong>','').replace('<br','')
    text = text.replace('Hide Full Comment', '').replace('Comment:','')
    info['Comment'] = text
    
    return info

def proceed2NextPage(url):
    num = int(url[url.find('pageIndex=')+10:url.rfind('&sortby')])
    newurl = url.replace('pageIndex={:g}'.format(num),
                         'pageIndex={:g}'.format(num+1))
    return newurl

def scrollReviews(reviewPage0, reviewsPerPage=5):
    # Going to the reviews page and finding the total number of reviews
    response = requests.get(reviewPage0)
    soup = BeautifulSoup(response.content, 'html.parser')

    content = soup.find('span', attrs={'class':'totalreviews'})
    if not content: # In case there are no reviews
        return None
    totalreviews = content.text
    totalreviews = totalreviews[:totalreviews.rfind('Total')-1]

    # Counting the number of pages
    totalreviews = int(totalreviews)
    num_pages = totalreviews // reviewsPerPage
    if totalreviews % reviewsPerPage: num_pages += 1
    
    # Iterating through pages and grabbing review data
    all_reviews = []
    for npage in range(num_pages):
        # Finding the relevant page
        if npage == 0:
            url = reviewPage0
        else:
            url = proceed2NextPage(url)
    
        # Get heading above userPosts
        response = requests.get(url)
        soup = BeautifulSoup(response.content, 'html.parser')
        content = soup.find('div', attrs={'id':'ratings_fmt'})
    
        revnum = 1
        for child in content.findChildren():
            if str(child).find('userPost') != -1:
                child_info = pull_review(child, revnum=revnum)
                all_reviews.append(child_info)
                revnum += 1
            
        time.sleep(1)
        
    # Process list all_reviews into dataframe and return
    return pd.DataFrame(all_reviews)
        
        
def getReviewsLink(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')

    content = soup.find('div', attrs={'class':'drug-review-lowest'})
    if content:
        searchLink = ''
        for child in content.findChildren():
            if str(child).find('"drug-review"') != -1:
                searchLink = str(child)
                searchLink = searchLink[searchLink.find('href="')+6:searchLink.rfind('">')]
                searchLink += '&pageIndex=0&sortby=3' # grabs reviews for every condition
                searchLink = 'https://www.webmd.com'+searchLink
                break
            
        return searchLink
    else:
        return False
    
def formatExactMed(s: str)->str:
    s = s.strip() # remove preceding and trailing white space
    s=s.replace(',','-') # Get rid of commas
    s=s.replace(' ', '-') # Get rid of white space
    s=s.replace('/', 'per') # remove things that make it look like a path
    return s

In [116]:
# Scrapes WebMD and pulls review for every medication it can
for ind, medication in zip(medsDF.index,medsDF['Medication']):
    for link, exactMed in zip(medsDF.loc[ind]['Parent links'], medsDF.loc[ind]['Exact medications']):
        if not glob.glob('Reviews/{:s}_reviews.csv'.format(formatExactMed(exactMed))):
            reviewLink = getReviewsLink(link)
        else:
            reviewLink = None

        if reviewLink:
            reviewsDF = scrollReviews(reviewLink)
            if type(reviewsDF) != type(None):
                reviewsDF.to_csv('Reviews/{:s}_reviews.csv'.format(formatExactMed(exactMed)), sep='$')


In [126]:
# Identifying missed medications
missed = []
files = glob.glob('Reviews/*_reviews.csv')
for med in medsDF['Medication']:
    allmissed = [True for file in files if file.lower().find(med.replace(' ','-').lower()) != -1]
    if not sum(allmissed): missed.append(med)
missed.remove('Clonidine Hydrochloride ')
        
#for miss in missed: print(miss)

# Hand grabbed because of weird webMD issue
review_pages = [
    'https://www.webmd.com/drugs/drugreview-4870-venlafaxine-oral.aspx?drugid=4870&drugname=venlafaxine-oral',
    'https://www.webmd.com/drugs/drugreview-22256-Focalin-oral.aspx?drugid=22256&drugname=Focalin-oral',
    'https://www.webmd.com/drugs/drugreview-7037-tenex.aspx?drugid=7037&drugname=tenex',
    'https://www.webmd.com/drugs/drugreview-6714-Catapres-oral.aspx?drugid=6714&drugname=Catapres-oral',
    'https://www.webmd.com/drugs/drugreview-151973-Aplenzin-oral.aspx?drugid=151973&drugname=Aplenzin-oral',
    'https://www.webmd.com/drugs/drugreview-76851-Wellbutrin-XL-oral.aspx?drugid=76851&drugname=Wellbutrin-XL-oral',
    'https://www.webmd.com/drugs/drugreview-1788-Depakote-oral.aspx?drugid=1788&drugname=Depakote-oral',
    'https://www.webmd.com/drugs/drugreview-6000-divalproex-oral.aspx?drugid=6000&drugname=divalproex-oral',
    'https://www.webmd.com/drugs/drugreview-13503-Tegretol-XR-oral.aspx?drugid=13503&drugname=Tegretol-XR-oral',
    'https://www.webmd.com/drugs/drugreview-1502-Tegretol-oral.aspx?drugid=1502&drugname=Tegretol-oral',
    'https://www.webmd.com/drugs/drugreview-8486-Lamictal-oral.aspx?drugid=8486&drugname=Lamictal-oral',
    'https://www.webmd.com/drugs/drugreview-4718-Seroquel-oral.aspx?drugid=4718&drugname=Seroquel-oral',
    'https://www.webmd.com/drugs/drugreview-6997-Prozac-oral.aspx?drugid=6997&drugname=Prozac-oral',
    'https://www.webmd.com/drugs/drugreview-19825-Sarafem-oral.aspx?drugid=19825&drugname=Sarafem-oral',
    'https://www.webmd.com/drugs/drugreview-35-Zoloft-oral.aspx?drugid=35&drugname=Zoloft-oral',
    'https://www.webmd.com/drugs/drugreview-6968-Paxil-oral.aspx?drugid=6968&drugname=Paxil-oral',
    'https://www.webmd.com/drugs/drugreview-91491-Cymbalta-oral.aspx?drugid=91491&drugname=Cymbalta-oral',
    'https://www.webmd.com/drugs/drugreview-13707-Remeron-oral.aspx?drugid=13707&drugname=Remeron-oral',
    'https://www.webmd.com/drugs/drugreview-64439-abilify.aspx?drugid=64439&drugname=abilify',
    'https://www.webmd.com/drugs/drugreview-9353-Nardil-oral.aspx?drugid=9353&drugname=Nardil-oral',
    'https://www.webmd.com/drugs/drugreview-95354-Emsam-transdermal.aspx?drugid=95354&drugname=Emsam-transdermal',
    'https://www.webmd.com/drugs/drugreview-152902-Saphris-sublingual.aspx?drugid=152902&drugname=Saphris-sublingual',
    'https://www.webmd.com/drugs/drugreview-1699-Zyprexa-oral.aspx?drugid=1699&drugname=Zyprexa-oral',
    'https://www.webmd.com/drugs/drugreview-146745-Invega-oral.aspx?drugid=146745&drugname=Invega-oral',
    'https://www.webmd.com/drugs/drugreview-9846-Risperdal-oral.aspx?drugid=9846&drugname=Risperdal-oral',
    'https://www.webmd.com/drugs/drugreview-20575-Geodon-oral.aspx?drugid=20575&drugname=Geodon-oral']

exactmeds = [s[s.rfind('=')+1:] for s in review_pages]

In [130]:
# Need to check pages for reviews that didn't have links ----> SCRAPING

# Adding two rows to account for a major medication in treatment guidelines that's not on NAMI
row_missing = {'Condition': 'Bipolar-Disorder',
               'Medication': 'Depakote',
               'Alternate names': 'Divalproex',
               'More common':'Acid or sour stomach; belching; body aches or pain; change in vision; congestion; continuing ringing or buzzing or other unexplained noise in the ears; hair loss or thinning of the hair; hearing loss; heartburn; impaired vision; lack or loss of strength; loss of memory; problems with memory; rash; seeing double; tender, swollen glands in the neck; trouble with swallowing; uncontrolled eye movements; voice changes; weight gain; weight loss',
               'Less common': 'Absent, missed, or irregular menstrual periods; back pain; burning, dry, or itching eyes; change in taste or bad unusual or unpleasant (after) taste; coin-shaped lesions on the skin; cough producing mucus; cramps; dandruff; discharge or excessive tearing; dry skin; earache; excess air or gas in the stomach or intestines; eye pain; feeling of constant movement of self or surroundings; full feeling; heavy bleeding; increased appetite; itching of the vagina or genital area; itching skin; loss of bowel control; neck pain; oily skin; pain; pain during sexual intercourse; pain or tenderness around the eyes and cheekbones ; passing gas; rash with flat lesions or small raised lesions on the skin; redness or swelling in the ear; redness, pain, swelling of the eye, eyelid, or inner lining of the eyelid; redness, swelling, or soreness of the tongue; sensation of spinning; sneezing; stiff neck; stopping of menstrual bleeding; thick, white vaginal discharge with no odor or with a mild odor',
               'Parent links': ['https://www.webmd.com/drugs/2/drug-1788/depakote-oral/details'],
               'Exact medications': ['Depakote-oral']
              }
medsDF = medsDF.append(row_missing, ignore_index=True)
row_missing = {'Condition': 'Bipolar-Disorder',
               'Medication': 'Divalproex',
               'Alternate names': 'Depakote',
               'More common':'Acid or sour stomach; belching; body aches or pain; change in vision; congestion; continuing ringing or buzzing or other unexplained noise in the ears; hair loss or thinning of the hair; hearing loss; heartburn; impaired vision; lack or loss of strength; loss of memory; problems with memory; rash; seeing double; tender, swollen glands in the neck; trouble with swallowing; uncontrolled eye movements; voice changes; weight gain; weight loss',
               'Less common': 'Absent, missed, or irregular menstrual periods; back pain; burning, dry, or itching eyes; change in taste or bad unusual or unpleasant (after) taste; coin-shaped lesions on the skin; cough producing mucus; cramps; dandruff; discharge or excessive tearing; dry skin; earache; excess air or gas in the stomach or intestines; eye pain; feeling of constant movement of self or surroundings; full feeling; heavy bleeding; increased appetite; itching of the vagina or genital area; itching skin; loss of bowel control; neck pain; oily skin; pain; pain during sexual intercourse; pain or tenderness around the eyes and cheekbones ; passing gas; rash with flat lesions or small raised lesions on the skin; redness or swelling in the ear; redness, pain, swelling of the eye, eyelid, or inner lining of the eyelid; redness, swelling, or soreness of the tongue; sensation of spinning; sneezing; stiff neck; stopping of menstrual bleeding; thick, white vaginal discharge with no odor or with a mild odor',
               'Parent links': ['https://www.webmd.com/drugs/2/drug-6000-8019/divalproex-oral/divalproex-sodium-enteric-coated-tablet-oral/details'],
               'Exact medications': ['divalproex-oral']}
medsDF = medsDF.append(row_missing, ignore_index=True)



In [ ]:
# Scraping relevant reviews
for reviewLink, exactMed in zip(review_pages, exactmeds):
    if not glob.glob('Reviews/{:s}_reviews.csv'.format(formatExactMed(exactMed))):
        reviewLink += '&pageIndex=0&sortby=3'
        reviewsDF = scrollReviews(reviewLink)
        if type(reviewsDF) != type(None):
            reviewsDF.to_csv('Reviews/{:s}_reviews.csv'.format(formatExactMed(exactMed)), sep='$')

In [ ]:
# Need to cluster reviews for each medication


In [67]:
num_reviews = []
for review in glob.glob('Reviews/*_reviews.csv'):
    df = pd.read_csv(review, sep='$')
    num_reviews.append((review, len(df.index)))

In [68]:
num_reviews

[('Reviews/Effexor-tablet_reviews.csv', 314),
 ('Reviews/paroxetine-ER_reviews.csv', 118),
 ('Reviews/guanfacine-HCL-ER_reviews.csv', 51),
 ('Reviews/aripiprazole-Suspension-Extended-Release-syringe_reviews.csv', 2),
 ('Reviews/clonidine-HCl-ER_reviews.csv', 191),
 ('Reviews/valproic-acid_reviews.csv', 20),
 ('Reviews/haloperidol-(bulk)-powder_reviews.csv', 3),
 ('Reviews/risperidone_reviews.csv', 65),
 ('Reviews/bupropion-XL_reviews.csv', 201),
 ('Reviews/fluoxetine-DR_reviews.csv', 203),
 ('Reviews/lamotrigine-ODT_reviews.csv', 174),
 ('Reviews/naltrexone-bupropion-tablet--extended-release_reviews.csv', 1),
 ('Reviews/haloperidol-lactate_reviews.csv', 1),
 ('Reviews/paroxetine-HCl_reviews.csv', 118),
 ('Reviews/aripiprazole_reviews.csv', 17),
 ('Reviews/valproic-acid-Solution_reviews.csv', 3),
 ('Reviews/folic-acid-vit-B6-vit-B12_reviews.csv', 1),
 ('Reviews/fluoxetine-(bulk)-100-%-powder_reviews.csv', 5),
 ('Reviews/amphetamine-Tablet-disintegrating-extended-release-biphasic-24-hr-(

In [83]:
np.is

Object `np.is` not found.
